In [ ]:
ORG = ''  # openai organization
KEY = ''  # openai key

# Utils

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install openai tiktoken mdextractor

In [ ]:
import os

def load_file(path):
  if path.endswith(".txt"):
      with open(path, 'r', encoding='utf-8') as file:
        return file.read().strip()
  raise Exception("Sorry, the file must be a txt")


def ask_gpt(prompt, model="gpt-4o-2024-08-06", max_tokens=100, temperature=0.4):
    import time
    from openai import OpenAI
    client = OpenAI(organization=ORG, api_key=KEY)
    try:
        time.sleep(10)
        response = client.chat.completions.create(
            model=model,
            messages = [{"role": "user", "content": x} for x in prompt],
            temperature=temperature,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
      print(e)


def extract_code(completion):
  from mdextractor import extract_md_blocks
  blocks = extract_md_blocks(completion)
  return blocks[0]

# Legal Fragments

In [ ]:
facts = """
issuing_member_state(MemberState) - this is the issuing member state
person_role(PersonId, Role) - the person has a certain role in the proceedings
offence_type(Offence) - this is the offence
offence_committed_in(MemberState) - the offence was committed in this member state
executing_member_state(MemberState) - this is the executing member state
executing_proceeding_purpose(PersonId, Purpose) - the proceedings in the executing member state are for a certain purpose
executing_proceeding(ExecutingMemberState, PersonId, Purpose) - the proceedings in the executing member state for this purpose relate to this person
person_age(PersonId, Age) - a person is a certain age
national_law_not_offence(Offence, MemberState) - an offence is not a crime according to the law of the execute member state
person_event(PersonId, SpecificEvent, Offence) - a person, who committed an offence, has been subject to a specific event during the course of the proceedings
issuing_proceeding_status(Offence, MemberState, SpecificStatus) - in the proceedings related to an offence in the issuing member state, this status applies
executing_proceeding_status(Offence, MemberState, SpecificStatus) - in the proceedings related to an offence in the executing member state, this status applies
person_nationality(PersonId, MemberState) - the person has the nationality of this state
person_residence(PersonId, MemberState) - the person has the residence of this state
age_of_criminal_responsibility(State, AgeLimit) - this is the age of criminal responsibility
"""

art_1_pl = """
eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence):-
    issuing_proceeding(IssuingMemberState, _, PersonId),
    offence_type(Offence),
    (
        art2_2applies(Offence)
    ;   art2_4applies(Offence)
    ),
    (
        executing_proceeding(ExecutingMemberState, PersonId, criminal_prosecution)
    ;   executing_proceeding(ExecutingMemberState, PersonId, execution_custodial_sentence)
    ;   executing_proceeding(ExecutingMemberState, PersonId, execution_detention_order)
    ).

executing_proceeding(ExecutingMemberState, PersonId, Purpose):-
    executing_member_state(ExecutingMemberState),
    executing_proceeding_purpose(PersonId, Purpose),
    member(Purpose, [criminal_prosecution, execution_custodial_sentence, execution_detention_order]),
    person_role(PersonId, subject_eaw).

issuing_proceeding(IssuingMemberState, _, PersonId):-
    issuing_member_state(IssuingMemberState),
    person_role(PersonId, subject_eaw).
"""

art_4_int = """
%%Article 4
%Grounds for optional non-execution of the European arrest warrant
%The executing judicial authority may refuse to execute the European arrest warrant:
"""

art_4_1 = """
%1. if, in one of the cases referred to in Article 2(4), the act on which the European arrest warrant is based does not constitute an offence under the law of the executing Member State; however, in relation to taxes or duties, customs and exchange, execution of the European arrest warrant shall not be refused on the ground that the law of the executing Member State does not impose the same kind of tax or duty or does not contain the same type of rules as regards taxes, duties and customs and exchange regulations as the law of the issuing Member State;
"""

art_4_1_pl = """
optional_refusal(article4_1, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    art2_4applies(Offence),
    national_law_not_offence(Offence, ExecutingMemberState).

%eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence) = the article concerns the european arrest warrant
%art2_4applies(Offence) = in one of the cases referred to in Article 2(4)
%national_law_not_offence(Offence, ExecutingMemberState) = the act [...] does not constitute an offence under the law of the executing Member State

"""

art_4_2 = """
%2. where the person who is the subject of the European arrest warrant is being prosecuted in the executing Member State for the same act as that on which the European arrest warrant is based;
"""

art_4_3 = """
%3. where the judicial authorities of the executing Member State have decided either not to prosecute for the offence on which the European arrest warrant is based or to halt proceedings, or where a final judgment has been passed upon the requested person in a Member State, in respect of the same acts, which prevents further proceedings;
"""

art_4_4 = """
%4. where the criminal prosecution or punishment of the requested person is statute-barred according to the law of the executing Member State and the acts fall within the jurisdiction of that Member State under its own criminal law;
"""

art_4_5 = """
%5. if the executing judicial authority is informed that the requested person has been finally judged by a third State in respect of the same acts provided that, where there has been sentence, the sentence has been served or is currently being served or may no longer be executed under the law of the sentencing country;
"""

art_4_6 = """
%6. if the European arrest warrant has been issued for the purposes of execution of a custodial sentence or detention order, where the requested person is staying in, or is a national or a resident of the executing Member State and that State undertakes to execute the sentence or detention order in accordance with its domestic law;
"""

art_4_7 = """
%7. where the European arrest warrant relates to offences which:
%(a) are regarded by the law of the executing Member State as having been committed in whole or in part in the territory of the executing Member State or in a place treated as such; or

%(b) have been committed outside the territory of the issuing Member State and the law of the executing Member State does not allow prosecution for the same offences when committed outside its territory.
"""

art_4_pl = """
optional_refusal(article4_2, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    person_event(PersonId, under_prosecution_by_executing_state, Offence). %proceeding_status(Offence, ExecutingMemberState, ongoing)

optional_refusal(article4_3, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    (
        executing_proceeding_status(Offence, ExecutingMemberState, no_prosecution)
    ;   executing_proceeding_status(Offence, ExecutingMemberState, halted)
    ;   person_event(PersonId, irrevocably_convicted_in_ms, Offence)  %proceeding_status(Offence, ExecutingMemberState, final_judgement),
    ).

optional_refusal(article4_4, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    executing_proceeding_status(Offence, ExecutingMemberState, statute_barred).

optional_refusal(article4_5, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    person_event(PersonId, irrevocably_convicted_in_third_state, Offence),
    %ExecutingMemberState \= ThirdState,
    (
        sentence_served_in_third_state(PersonId)
    ;   sentence_being_served_in_third_state(PersonId)
    ;   sentence_execution_impossible_in_third_state(PersonId)
    ).

optional_refusal(article4_7_a, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    executing_proceeding_status(Offence, ExecutingMemberState, committed_inside_territory).


optional_refusal(article4_7_b, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    issuing_proceeding_status(Offence, IssuingMemberState, committed_outside_territory),
    prosecution_not_allowed(Offence, ExecutingMemberState).

"""

art_4_6_pl ="""
optional_refusal(article4_6, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    (
        executing_proceeding(ExecutingMemberState, PersonId, execution_custodial_sentence)
    ;   executing_proceeding(ExecutingMemberState, PersonId, execution_detention_order)
    ),
    (
%        person_staying_in(PersonId, ExecutingMemberState)
        person_nationality(PersonId, ExecutingMemberState)
    ;   person_residence(PersonId, _, ExecutingMemberState, _)
    ).
"""

case_example = """
%%% Case Law Name
case_law(EU Article, Case Law Name):-
    Case Law Elements.

%%% Prolog Article
case_law_refusal(EU Article, Case Law Name, europeanArrestWarrant):-
    optional_refusal(EU Article, ExecutingMemberState, europeanArrestWarrant),
    case_law(EU Article, Case Law Name).
"""

In [ ]:
sentenza_4_6 = load_file("/content/sentenza_4_6_eaw.txt")

# Task 1: Article Genaration

## Prompts

In [ ]:
def complete_from_facts(code_base, fact, target, temperature=0.1, max_tokens=2000):
    txt = f"""
        Given the following list of Prolog facts and rules:

        List of Prolog Rules:

        {code_base}

        List of Prolog Facts:

        {fact}

        Complete the representation of the following article:

        {target}

        Only use the Prolog facts provided.
        New rules can be created, as long as they are based on existing facts.

        The output should follow the following structure:

        fragment of text 1
        prolog representation 1

        ...

        fragment of text n
        prolog representation n

        The goal of the rules is always the mandatory / optional refusal of the instrument.
        Your output should contain only the required code inserted in a Markdown code block.
    """
    return ask_gpt([txt], temperature=temperature, max_tokens=max_tokens)


def revise_facts(code_base, temperature=0.5, max_tokens=2000):
    txt = f"""
        Given the following list of Prolog rules:

        {code_base}

        Identify possible problems:
        - inconsistent naming or arity.
        - fail to capture relevant information.
        - no correspondance between natural language and rules.
        - mixed legal conditions.

        The output should include only the identified problems.

        The output should follow the following structure:
        problem 1
        explanation of the problem
        ...
        problem n
        explanation of the problem

        Your output should contain only the required code inserted in a Markdown code block.
    """
    return ask_gpt([txt], temperature=temperature, max_tokens=max_tokens)


def apply_revision(code_base, revision, numbers, temperature=0.1, max_tokens=2000):

    problems = extract_code(revision).split("\n\n")
    selected_problems = [y for x, y in enumerate(problems) if x + 1 in numbers]

    txt = f"""
        Given the following list of Prolog rules and a set of revisions:

        List of Prolog Rules:

        {code_base}

        Revision:

        {selected_problems}

        Apply the revisions and solve the issues!

        Your output should contain only the required code inserted in a Markdown code block.
    """
    return ask_gpt([txt], temperature=temperature, max_tokens=max_tokens)

## Article 4 - Complete

In [ ]:
target = [art_4_2, art_4_3, art_4_4, art_4_5, art_4_6, art_4_7]

completion = [complete_from_facts(f"""
        {art_4_int}
        {art_4_1}
        {art_1_pl}
        {art_4_1_pl}
  """, facts, x) for x in target]

for x in completion:
  print(x)
  print()

```prolog
%2. where the person who is the subject of the European arrest warrant is being prosecuted in the executing Member State for the same act as that on which the European arrest warrant is based;

optional_refusal(article4_2, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    executing_proceeding(ExecutingMemberState, PersonId, criminal_prosecution),
    person_event(PersonId, prosecution, Offence).
```

```prolog
%3. where the judicial authorities of the executing Member State have decided either not to prosecute for the offence on which the European arrest warrant is based
optional_refusal(article4_3a, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    executing_proceeding_status(Offence, ExecutingMemberState, not_to_prosecute).

%or to halt proceedings,
optional_refusal(article4_3b, ExecutingMemberState, europeanArrestWarran

In [ ]:
s_compl = "\n\n\n".join([extract_code(x) for x in completion])

revision = revise_facts(f"""
        {art_4_int}
        {art_4_1}
        {art_1_pl}
        {art_4_1_pl}
        {s_compl}
  """)
print(revision)

```markdown
problem 1
The rule `jurisdiction_under_criminal_llaw/2` has a typo in the predicate name. It should be `jurisdiction_under_criminal_law/2`.

problem 2
The rule `jurisdiction_under_criminal_llaw/2` does not match the predicate used in `optional_refusal(article4_4, ...)`.

problem 3
The rule `optional_refusal(article4_7b, ...)` checks for `\+ executing_proceeding(ExecutingMemberState, PersonId, criminal_prosecution)`, but the condition in the natural language description is about the law not allowing prosecution, not the absence of prosecution.

problem 4
The rule `optional_refusal(article4_3c, ...)` uses `executing_proceeding_status(Offence, ExecutingMemberState, prevents_further_proceedings)` to check if further proceedings are prevented, but this condition is not clearly defined in the natural language description.

problem 5
The rule `state_undertakes_execution/3` does not capture the condition that the state undertakes to execute the sentence or detention order "in accor

In [ ]:
revised = apply_revision(f"""
        {art_4_int}
        {art_4_1}
        {art_1_pl}
        {art_4_1_pl}
        {s_compl}
  """, revision, [1,2,3])
print(revised)

```prolog
% Corrected rule with the typo fixed
jurisdiction_under_criminal_law(ExecutingMemberState, Offence):-
    offence_committed_in(ExecutingMemberState).

% Corrected rule for optional refusal under article 4.7b
optional_refusal(article4_7b, ExecutingMemberState, europeanArrestWarrant):-
    eaw_matter(PersonId, IssuingMemberState, ExecutingMemberState, Offence),
    \+ offence_committed_in(IssuingMemberState),
    \+ law_allows_prosecution_outside_territory(ExecutingMemberState, Offence).

% New rule to check if the law allows prosecution outside its territory
law_allows_prosecution_outside_territory(ExecutingMemberState, Offence):-
    executing_member_state(ExecutingMemberState),
    offence_type(Offence),
    allows_prosecution_outside_territory(ExecutingMemberState, Offence).
```


## Article 4(6) - Paper Example

In [ ]:
target = [art_4_6]

completion = [complete_from_facts(f"""
        {art_4_int}
        {art_4_1}
        {art_1_pl}
        {art_4_1_pl}
  """, facts + "\nperson_staying_in(PersonId, MemberState) - the person lives in this state", x, temperature=0.1) for x in target]

for x in completion:
  print(x)
  print()

```prolog
% if the European arrest warrant has been issued for the purposes of execution of a custodial sentence or detention order,
eaw_execution_purpose(PersonId, ExecutingMemberState):-
    executing_proceeding(ExecutingMemberState, PersonId, execution_custodial_sentence)
    ; executing_proceeding(ExecutingMemberState, PersonId, execution_detention_order).

% where the requested person is staying in, or is a national or a resident of the executing Member State
person_in_executing_state(PersonId, ExecutingMemberState):-
    person_staying_in(PersonId, ExecutingMemberState)
    ; person_nationality(PersonId, ExecutingMemberState)
    ; person_residence(PersonId, ExecutingMemberState).

% and that State undertakes to execute the sentence or detention order in accordance with its domestic law
state_undertakes_execution(ExecutingMemberState, Offence):-
    executing_proceeding_status(Offence, ExecutingMemberState, execution_undertaken).

% optional refusal of the European arrest warrant

In [ ]:
s_compl = "\n\n\n".join([extract_code(x) for x in completion])

revision = revise_facts_2(f"""
        {art_4_int}
        {art_4_1}
        {art_1_pl}
        {art_4_1_pl}
        {s_compl}
  """)
print(revision)

```markdown
problem 1
Inconsistent naming: The predicate `eaw_for_execution/2` is named to suggest it checks if an EAW is for execution purposes, but its implementation checks if there is an executing proceeding for a custodial sentence or detention order. This could be misleading.

problem 2
Inconsistent naming: The predicate `executing_proceeding/3` includes a parameter named `Purpose`, but the rule's logic does not clearly distinguish between purposes, as it uses the same predicate to check for different types of proceedings. This could lead to confusion.

problem 3
Fail to capture relevant information: The rule `optional_refusal(article4_1, ExecutingMemberState, europeanArrestWarrant)` does not consider the exceptions related to taxes or duties, customs, and exchange regulations when determining if the EAW can be refused.

problem 4
No correspondence between natural language and rules: The rule `optional_refusal(article4_6, ExecutingMemberState, europeanArrestWarrant)` does not exp

In [ ]:
revised = apply_revision_2(f"""
        {art_4_int}
        {art_4_1}
        {art_1_pl}
        {art_4_1_pl}
        {s_compl}
  """, revision, [5])
print(revised)

```prolog
% Revised predicate to better represent the legal condition of a state undertaking to execute a sentence according to its domestic law
state_undertakes_execution(ExecutingMemberState, Offence):-
    domestic_law_execution_commitment(ExecutingMemberState, Offence).

% New predicate to represent the commitment of the executing state to undertake execution according to its domestic law
domestic_law_execution_commitment(ExecutingMemberState, Offence):-
    executing_member_state(ExecutingMemberState),
    offence_type(Offence),
    member(Offence, [offence1, offence2, offence3]). % Replace with actual offences that the state commits to execute
```



# Task 2: Case Generation

## Prompts

In [ ]:
def case_law_summary(target, temperature=0.1, max_tokens=2000):
    txt = f"""
        Provide a list of conditions related to the application
          of a ground of refusal according to European Law.
        Here is the case law:
        {target}

        The output should be in the following format:
        - Case Law Name
          - ground of refusal article
          - ground of refusal conditions
          - explanation of the elements
          - case law influences (Apply only the elements which add information to existing elements.
             Do not consider elements which expand the scope of the article for the legislator.)


    """
    return ask_gpt([txt], temperature=temperature, max_tokens=max_tokens)

def case_law_rules(target, code_base, article, temperature=0.5, max_tokens=2000):
    txt = f"""
        Here are a list of elements that change the application of an article based on case law.
        The list comes with an explanation for each element.
        {target}

        Your task is to create new rules which formalize the case law.
        Use only the elements necessary for applying of the ground of refusal.
        Follow this style:
        {code_base}

        This is the article which relates to the case:
        {article}

        Apply only the elements which add information to existing predicates.
        Do not consider elements which expand the scope of the article for the legislator.

        These rules shall contain only the relevant case law information.

        The final output should be in the following format:
        - Article applied
        - New prolog rule
        - Explanation of the conditions

    """
    return ask_gpt([txt], temperature=temperature, max_tokens=max_tokens)

## Judgment of the Court (Grand Chamber), 17 July 2008, Case C-66/08 - Paper Example

In [ ]:
summary = case_law_summary(sentenza_4_6)
print(summary)

- **Case Law Name**
  - KOZŁOWSKI

- **Ground of Refusal Article**
  - Article 4(6) of Council Framework Decision 2002/584/JHA

- **Ground of Refusal Conditions**
  - The requested person is staying in, or is a national or a resident of, the executing Member State.
  - The executing Member State undertakes to execute the sentence or detention order in accordance with its domestic law.

- **Explanation of the Elements**
  - **Staying**: The term "staying" refers to a situation where the requested person, following a stable period of presence in the executing Member State, has acquired connections with that State which are of a similar degree to those resulting from residence. This includes an overall assessment of various objective factors such as the length, nature, and conditions of the person's presence, as well as their family and economic connections with the executing Member State.
  - **Resident**: A person is considered a "resident" in the executing Member State when they have e

In [ ]:
article4_6 = """
  % if the European arrest warrant has been issued for the purposes of execution of a custodial sentence or detention order,
  eaw_execution_purpose(PersonId, ExecutingMemberState):-
      executing_proceeding(ExecutingMemberState, PersonId, execution_custodial_sentence)
      ; executing_proceeding(ExecutingMemberState, PersonId, execution_detention_order).

  % where the requested person is staying in, or is a national or a resident of the executing Member State
  person_in_executing_state(PersonId, ExecutingMemberState):-
      person_staying_in(PersonId, ExecutingMemberState)
      ; person_nationality(PersonId, ExecutingMemberState)
      ; person_residence(PersonId, ExecutingMemberState).

  % and that State undertakes to execute the sentence or detention order in accordance with its domestic law
  state_undertakes_execution(ExecutingMemberState, Offence):-
      executing_proceeding_status(Offence, ExecutingMemberState, execution_undertaken).

  % optional refusal of the European arrest warrant
  optional_refusal(article4_6, ExecutingMemberState, europeanArrestWarrant):-
      eaw_execution_purpose(PersonId, ExecutingMemberState),
      person_in_executing_state(PersonId, ExecutingMemberState),
      state_undertakes_execution(ExecutingMemberState, Offence).
"""

summary_portion = """
- **Case Law Influences**
  - **Overall Assessment**: The executing judicial authority must make an overall assessment of various objective factors characterizing the situation of the requested person, including the length, nature, and conditions of their presence, and the family and economic connections they have with the executing Member State.
  - **Non-Interruption and Compliance**: The fact that the requested person’s stay in the executing Member State was not uninterrupted and did not comply with the national legislation on residence of foreign nationals can be relevant but not conclusive factors in determining whether the person is "staying" in the executing Member State.
  - **Criminal Activity and Detention**: The fact that the requested person systematically commits crimes in the executing Member State or is in detention there serving a custodial sentence is not relevant for determining whether they are "staying" in the executing Member State but may be relevant for assessing whether there are grounds for not implementing a European arrest warrant.
"""

rules = case_law_rules(summary_portion, case_example, article4_6)
print(rules)

### Article Applied
Article 4(6) of the European Arrest Warrant Framework Decision

### New Prolog Rule
```prolog
%%% Case Law Name
case_law(article4_6, CaseLawName):-
    overall_assessment(PersonId, ExecutingMemberState),
    non_interruption_and_compliance(PersonId, ExecutingMemberState).

%%% Prolog Article
case_law_refusal(article4_6, CaseLawName, europeanArrestWarrant):-
    optional_refusal(article4_6, ExecutingMemberState, europeanArrestWarrant),
    case_law(article4_6, CaseLawName).

% overall_assessment(PersonId, ExecutingMemberState) - overall assessment of the requested person's situation
overall_assessment(PersonId, ExecutingMemberState):-
    person_presence(PersonId, ExecutingMemberState, Length, Nature, Conditions),
    family_connections(PersonId, ExecutingMemberState),
    economic_connections(PersonId, ExecutingMemberState).

% non_interruption_and_compliance(PersonId, ExecutingMemberState) - assessment of the continuity and legality of the person's stay
non_interru